In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [17]:
df = pd.read_csv("/content/housing.csv")
df[:3]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


In [18]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [20]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [21]:
def map_categorical_to_numerical(categorical_data):
    unique_categories = set(categorical_data)
    mapping = {category: i for i, category in enumerate(unique_categories)}
    return mapping

In [22]:
ocean_proximity_numerical=map_categorical_to_numerical(df['ocean_proximity'])
ocean_proximity_numerical

{'ISLAND': 0, 'NEAR OCEAN': 1, 'INLAND': 2, 'NEAR BAY': 3, '<1H OCEAN': 4}

In [23]:
new_ocean_proximity = np.array([])

for ocean_proximity in df['ocean_proximity']:
  new_ocean_proximity = np.append(new_ocean_proximity,ocean_proximity_numerical[ocean_proximity])

new_ocean_proximity[:3]

array([3., 3., 3.])

In [24]:
df = df.drop("ocean_proximity",axis=1)
df["ocean_proximity"] = new_ocean_proximity
df[:5]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3.0


In [25]:
df = df.fillna(df.mean())
df[:4]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3.0


In [26]:
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols_to_normalize = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']
df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
df[:3]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0.211155,0.567481,0.784314,0.022331,0.019863,0.008941,0.020556,0.539668,452600.0,0.75
1,0.212151,0.565356,0.392157,0.180503,0.171477,0.067210,0.186976,0.538027,358500.0,0.75
2,0.210159,0.564293,1.000000,0.037260,0.029330,0.013818,0.028943,0.466028,352100.0,0.75


In [28]:
y = df["median_house_value"]
y = y/1000
X = df.drop("median_house_value",axis=1)

X.shape, y.shape

((20640, 9), (20640,))

In [29]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [30]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(4),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mae",
              optimizer=tf.keras.optimizers.Adam(),
              metrics = ["mse"])

model.fit(X_train,y_train,
          epochs=100,
          verbose=1)

Epoch 1/100
452/452 [==============================] - 3s 3ms/step - loss: 79.1719 - mse: 12845.4355
Epoch 2/100
452/452 [==============================] - 2s 3ms/step - loss: 54.4245 - mse: 6086.0195
Epoch 3/100
452/452 [==============================] - 4s 8ms/step - loss: 52.6916 - mse: 5729.0645
Epoch 4/100
452/452 [==============================] - 4s 9ms/step - loss: 51.6072 - mse: 5489.6440
Epoch 5/100
452/452 [==============================] - 3s 6ms/step - loss: 50.9792 - mse: 5316.7671
Epoch 6/100
452/452 [==============================] - 1s 3ms/step - loss: 50.2986 - mse: 5184.0303
Epoch 7/100
452/452 [==============================] - 2s 3ms/step - loss: 50.4173 - mse: 5193.4097
Epoch 8/100
452/452 [==============================] - 1s 3ms/step - loss: 50.4324 - mse: 5188.1318
Epoch 9/100
452/452 [==============================] - 2s 4ms/step - loss: 50.4435 - mse: 5191.3618
Epoch 10/100
452/452 [==============================] - 2s 5ms/step - loss: 50.0566 - mse: 5118.538

In [31]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
reg.fit(X_train, y_train)

RandomForestRegressor()

In [32]:
y_pred = reg.predict(X_test)

In [33]:
from sklearn import metrics
acc_rf = metrics.r2_score(y_test, y_pred)
print('MAE:',metrics.mean_absolute_error(y_test, y_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 32.552660875323014
MSE: 2527.158401579369
RMSE: 50.27085041631352
